In [ ]:
import pandas as pd 

In [ ]:
df=pd.read_csv("/content/Ham and Spam Data.csv",sep="\t",names=["label","Message"])
df

,label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df["Message"].loc[110]

"Dont worry. I guess he's busy."

In [ ]:
import re
import nltk
nltk.download("stopwords","en")

[nltk_data] Downloading package stopwords to en...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
word="maining"
print(ps.stem(word))

main


In [ ]:
stopword=list(stopwords.words("english"))

In [ ]:

def data_preprocessing(dataframe):
  training_dataset=[]
  
  for i in range(len(dataframe)):
    review=re.sub("[^a-zA-Z]"," ",dataframe["Message"][i])
    review=review.lower()
    
    review=review.split()
    review=[ps.stem(word) for word in review if not word  in stopword]
    review=" ".join(review)
    training_dataset.append(review)
  
  return training_dataset

corpus=data_preprocessing(df)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(max_features=2500,binary=True)
x=cv.fit_transform(corpus)
x=x.toarray()
print(x[1])
print(x.shape)

[0 0 0 ... 0 0 0]
(5572, 2500)


In [ ]:
df

,label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
y=pd.get_dummies(df["label"])
y=y.iloc[:,1].values

[0 0 1 ... 0 0 0]


In [ ]:
print(x.shape[0]==y.shape[0])

True


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,shuffle=True,test_size=0.2)

In [ ]:
print(x_train.shape)
print(y_train.shape)

(4457, 2500)
(4457,)


In [ ]:
from sklearn.naive_bayes import  MultinomialNB

In [ ]:
detection_model=MultinomialNB().fit(x_train,y_train)


In [ ]:
import numpy as np 
y_pred=detection_model.predict(x_test)
print(np.unique(y_pred,return_counts=True))

(array([0, 1], dtype=uint8), array([967, 148]))


In [ ]:

from sklearn.metrics import accuracy_score, classification_report 

In [ ]:
print(accuracy_score(y_test,y_pred))

print(classification_report(y_test,y_pred))

0.9928251121076234
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       965
           1       0.98      0.97      0.97       150

    accuracy                           0.99      1115
   macro avg       0.99      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



Use TF-IDF for word to vect conversion, following implementation is usinf TF-IDF.
 Bigrams is also one of the technique which is being used to improve the accuracy of model to keep overall meaning of text

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

cv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
x=cv.fit_transform(corpus)
x=x.toarray()
print(x[1])
print(x.shape)

[0. 0. 0. ... 0. 0. 0.]
(5572, 2500)


In [ ]:
detection_model_TDIDF=MultinomialNB().fit(x_train,y_train)
y_pred_TFIDF=detection_model_TDIDF.predict(x_test)
print(accuracy_score(y_test,y_pred_TFIDF))

0.9874439461883409


USE Randomforest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred_random=classifier.predict(x_test)

print(accuracy_score(y_test,y_pred_random))

0.9838565022421525
